# Optical flow for control
In this notebook, we will study how we can determine ego-motion and structure with the help of optical flow from an image sequence. We assume a pinhole camera in this notebook. We first start with a theoretical derivation of the optical flow formulas (expressing the optical flow in terms of ego-motion and structure of the observed scene). Then we will discuss how structure and ego-motion can be obtained from these formulas. Finally, you can play with a Python script that determines important optical flow observables such as the divergence from an image sequence.

## Deriving the optical flow formula
Each point in an image $(x, y)$ has an associated optical flow $(u, v)$. Here we will derive an expression for the optical flow in terms of the movement of the camera, translational velocities $(U, V, W)$ and rotation rates $(A,B,C)$, and the coordinate of the projected point in the real world, $(X, Y, Z)$. This derivation is based on [2], from which we adopt the notation.

We also use the coordinate system as introduced in [2], which is shown in Figure 1. In the system, O represents the center of the camera, $X$ is positive toward the left, $Y$ positive up, and $Z$ is positive in the direction of the camera’s principal axis.

<CENTER>
<IMG SRC="coordinate_system.png" ALT="Coordinate system used in the optical flow derivation."></IMG>
<small>Figure 1: Coordinate system from [2].</small>
</CENTER>

We start with the formulas for the image coordinate of a world point $P$ at $(X, Y, Z)$:

$x = \frac{X}{Z}$ and $y = \frac{Y}{Z}$,

where the reader should note that $x$ and $y$ are _normalized_ image coordinates. The optical flow then is:

$u = \frac{\partial}{\partial t} x = \dot{x}$

and

$v = \frac{\partial}{\partial t} y = \dot{y}$

Here, we first express $\dot{x}$ and $\dot{y}$ in the variables representing the camera movement and $P$'s coordinate:

$\dot{x} = \frac{\dot{X}}{Z} - \frac{X \dot{Z}}{Z^2}$      (Eq. 1),

and

$\dot{y} = \frac{\dot{Y}}{Z} - \frac{Y \dot{Z}}{Z^2}$      (Eq. 2),

which shows that we should find the expressions for $\dot{X}$, $\dot{Y}$ and $\dot{Z}$.

We will determine the time derivative of $x$, as the derivation of $\dot{y}$ is similar. Let us start with $\dot{X}$, which captures the change in coordinate $X$ as measured from the camera's origin over time. Obviously, if the camera has a positive velocity $U$, in its own reference frame the (static) point $P$ will move with the same velocity in the other direction. So:

$\dot{X} = -U + \ldots$

<CENTER>
<IMG SRC="rotation_beta.png" SIZE="200" ALT="Effect of a rotation around the Y-axis on the X-position of a point P."></IMG>
<small>Figure 2: Rotation around Y -axis, with rotation rate B.</small>
</CENTER>

Furthermore, a rotation around the $Y$-axis ($B$) also has an effect on the change in coordinate $X$. Please look at Figure 2, depicting a point P that is rotated with a rotation rate $B$. First, we can express $X$ and $Z$ in terms of the angle $\beta$ and distance of P to the origin, $R$:

$X = R \mathrm{cos}(\beta)$, and $Z = R \mathrm{sin}(\beta)$.

Then we can determine $\dot{X}$ as:

$\dot{X} = \dot{R} \mathrm{cos}(\beta) - R \mathrm{sin}(\beta) \dot{\beta}$,

where $\dot{R}=0$ for a rotation, and $\dot{\beta} = B$:

$\dot{X} = - R \mathrm{sin}(\beta) B = -B Z$,

where the last equality uses the equality $Z = R \mathrm{sin}(\beta)$. So this brings us to:

$\dot{X} = -U -BZ + \ldots$

A similar reasoning (left as exercise for the student) leads to the addition of $+CY$. The derivations for $\dot{Y}$ are also similar, with as result:

$\dot{X} = -U -BZ + CY$,
$\dot{Y} = -V -CX + AZ$,
$\dot{Z} = -W -AY + BX$.

We can fill in these expressions into Eq. 1 and 2:

$u = \dot{x} = \frac{-U -BZ + CY}{Z} - \frac{X (-W -AY + BX)}{Z^2}$,

which we can write differently as:

$u = \dot{x} = -\frac{U}{Z} - B + Cy + x \frac{W}{Z} + A x y - B x^2$,

where we have used $x = \frac{X}{Z}$ and $y = \frac{Y}{Z}$.

In [2], the optical flow formulas are finally written as follows:

$u = (-\frac{U}{Z} - B + Cy) - x (\frac{W}{Z} - A y + B x)$,

$v = (-\frac{V}{Z} -Cx + A) -y (-\frac{W}{Z} - A y + B x)$.

## Information available from optical flow